# GPU 编程和 CUDA 入门

学习 GPU/CUDA 编程必须要有 C 语言基础（尤其是内存分配和指针），使用 python 进行 CUDA 编程必须有 python 基础和 numpy 包的使用经验。

## 最直观的 GPU/CUDA 介绍

GPU 最常用的功能是图象处理，我们知道图象是一个二维数组 img\[X\]\[Y\]，数组下标即图象在这一点的颜色。

例如，img\[50\]\[100\] = RBG(255,0,0)，表示坐标(50, 100)位置处颜色为红色。（RGB是一种颜色标识方式）

### 试想一下，我们需要将图象放大，显示器的每个像素点的颜色将怎么变化？

为了简单起见，不妨设放大中心为 (0,0)，即放大过程中原点的位置不变，那么放大过程很简单，即

new_img\[i\]\[j\]=old_img\[i/c\]\[j/c\]

其中 c 是放大倍数，这里忽略计算中小数产生的误差。（会导致锯齿、丢失信息）

以上操作在 CPU 编程中需要写出这样的代码

```C
for(int i = 0;i < width * c;i++)
    for(int j = 0;j< height * c; j++)
        new_img[i][j] = old_img[i/c][j/c];
```

可以看到代码的主体是一个双重循环，如果缩放一个上亿像素（长*宽≈1亿）的图片，再加上现在用户操作强调流畅性，即要求屏幕显示图片被放大的动态过程，主频 GHz 的 CPU 运行起来很明显会出现卡顿。

**在 GPU/CUDA 中，同样的功能可以不用循环，交由不同的线程完成**

```C
int i = threadIdx.x; // 获取线程 x 方向编号
int j = threadIdx.y; // 获取线程 y 方向编号
new_img[i][j] = old_img[i/c][j/c];
```

代码执行的逻辑是这样的：**事先给 GPU 中的线程编号，编号可以是1维，也可以2维或3维，这里很明显是2维编号，即每个线程的编号是一个二元数对（i,j），这些线程同时执行上面的三横代码，互相分工，完成图片的缩放。**

其中代码中 threadIdx.x 和 threadIdx.y 获取线程 2 个维度上的编号。

例如编号（0, 0）的线程，他执行的代码是 new_img\[0\]\[0\]=old_img\[0/c\]\[0/c\]

编号（100, 200）的线程，执行的代码是 new_img\[100\]\[200\]=old_img\[100/c\]\[200/c\]

### 总结 CPU 和 GPU 的不同

1. CPU 和 GPU 都由线程组成，CPU 线程少（一般4-16个），GPU 线程多（>1000）

2. CPU 的线程，**互相之间完全独立**。一边看文献一边写论文，这两件事在 CPU 中可以交给不同的线程完成，但是 GPU 中线程不独立，它们必须完成相同的功能（执行相同的代码），就像上面的图片放大的例子一样。

3. GPU 编程的关键即把任务拆分。

## 开发环境搭建

### visual studio

安装 visual studio，任意版本都可以，目的是得到一个 C/C++ 编译器 cl.exe。安装时选择安装“使用C++的桌面开发”。安装后，在安装目录搜索 cl.exe，典型目录为 

`C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.27.29110\bin\Hostx64\x64` 

将这个目录添加到环境变量 path 中。添加完毕后，检查

<img src="./img/A01CL编译器.jpg" width=40%></img>

### CUDA 开发包

1. 检查电脑是否使用 NVIDIA 显卡。（可以去设备管理器查看）

2. 进入 NVIDIA Control Panel 查看 CUDA 版本

依次点即“系统信息”，在弹窗中点即“组件”，查看 NVCUDA64.DLL 文件，如下图所示。

产品名称一栏显示的 CUDA 版本为 11.1.114，记住这个版本号。（因为等会需要下载配套的 CUDA 开发包）

<img src="./img/A02CUDA版本.jpg" width=70%></img>

3. （可选）升级显卡驱动

升级显卡驱动，第 2 步中的 CUDA 版本也会升级。这样就不用费力下载早期的 CUDA 开发包。

升级后，重新按照第 2 步查看 CUDA 版本。

4. 下载安装 CUDA 开发包

https://developer.nvidia.com/zh-cn/cuda-downloads

NVIDIA 官问提供 CUDA 开发包下载，开发包的版本必须和 2 步中看到的版本一致。

最新版可以直接下载，早期版本见 https://developer.nvidia.com/cuda-toolkit-archive

按照下图，选择系统 win 位数 64 版本 10，下载本地安装包。大小为 3G 左右。

<img src="./img/A03CUDA下载.jpg" width=70%></img>

下载完毕后，全部默认安装即可。

5. python 用户安装 pycuda

`pip install pycuda`

### 检查环境配置是否成功

一般来说，只要第 5 步 pycuda 安装成功，就说明环境配置成功了。

如果没有执行第 5 步，在 cmd 中输入 `nvcc --version`，将显示 CUDA 程序编译器 nvcc 的版本。

<img src="./img/A04NVCC.jpg" width=40%></img>

注释：nvcc 编译器只处理 GPU 执行相应的代码，代码部分中 CPU 部分，会自动交给 cl 编译器执行（所以要安装 visual studio）。






## Hello World 程序

在纯 C 语言中，我们定义一个打印 Hello World 的函数，并在 main 函数中调用它。

```C
#include <stdio.h>

void greet(){
    printf("Hello World\n");
}

int main(){
    greet();
    return 0;
}
```

如果要让 greet() 函数在 GPU 中运行，我们需要两步操作：

-  指明 great() 函数的调用方式，在函数定义行的最前面加上 `__global__`，这表示**函数由 CPU 调用并在 GPU 上执行**。

`__global__` 是 CUDA 代码中的一个关键字，它定义了函数的调用者和执行者。类似的关键字还有`__device__`，表示**函数由 GPU 调用并在 GPU 上执行。**

-  指明 great() 函数执行时需要多少个线程，以及线程编号方式。在 main 函数中执行 great() 函数的语句是`greet();`，只要在函数名和后面的括号中插入`<<<dim3(dx,dy,dz) ,dim3(tx,ty,tz)>>>`即可。

为了解释`<<<dim3(dx,dy,dz) ,dim3(tx,ty,tz)>>>`的意义，首先介绍 GPU 中线程的编号方式。因为 GPU 中线程数目很多，因此采用分层思想（就像学生们不是都在一个班，而是分布在不同班级），线程中的“班级”称为**线程块**，一个 GPU 中存在若干个**线程块**，**线程块**内存在若干个**线程**，这里的具体数目可以任意指定，不是固定的。

另外**线程块**和块内的**线程**编号，可以采用一维、二维、三维的形式。

示例一：`<<<dim3(1),dim3(1)>>>`表示函数执行时，启动 1 个**线程块**，每个**线程块**内 1 个**线程**，一维可以简写为 <<<1,1>>>

示例一：`<<<dim3(2,3,4) ,dim3(1,5,1)>>>`表示函数执行时，启动 2\*3\*4 个**线程块**（共 24 个），每个**线程块**内 1\*5\*1 个**线程**。（共 5 个）

修改后的程序如下：

```C
// hello.cu
#include <stdio.h>

 __global__ void greet(){
    printf("Hello World\n");
}

int main(){
    greet<<<1,1>>>();
    return 0;
}
```

1. 将文件保存为 `hello.cu`，cu 表示 CUDA 程序源代码。

<img src="./img/A05nvcc01.jpg" width=70%></img>


2. 在该目录下输入命令`nvcc hello.cu`。编译成功后，按照 C 编译器的传统输出`a.exe`文件。（当然可以提前指定输出文件名，使用 -o 命令）

<img src="./img/A05nvcc02.jpg" width=70%></img>

3. 运行`a.exe`，查看执行结果

<img src="./img/A05nvcc03.jpg" width=70%></img>

至此 CUDA 版的 hello World 编写执行成功！

**试试看** 将源代码中`<<<1,1>>>`，修改为`<<<1,2>>>`，再编译执行，看看会发生什么？再改为`<<<2,2>>>`呢？和你的猜想一致吗？

## CUDA 编程核心知识点

### 线程编号

在之前的 Hello World 程序中，如果将线程分配改为 `<<<1,2>>>` 或 `<<<2,2>>>`，将打印出 2 个和 4 个 Hello World 语句。

这一小节利用 CUDA 程序中的内置变量 `blockIdx` 和 `threadIdx`，让程序可以运行时知道自己正在被哪个线程执行。

下面的程序 whoami 将打印出线程组 id 和线程 id。

```C
#include <stdio.h>

 __global__ void report(){
    int i = blockIdx.x;
    int j = threadIdx.x;

    printf("My group id is %d, and my thread id is %d\n",i,j);
}

int main(){
    report<<<3,2>>>();
    return 0;
}
```

运行结果如下图。

<img src="./img/A06WhoAmI.jpg" width=70%></img>

可以清楚的看到 `<<<3,2>>>` 中尖括号第一个数字 3 表示一共有 3 个线程组，且编号为 0,1,2。尖括号内第二个数字 2 表示每组内都有 2 个线程，编号是 0 和 1。

另外，多线程执行是乱序的，图中最先打印的是 2 号线程组，然后是 0 号，最后是 1 号。

`blockIdx` 和 `threadIdx` 是三维变量，这是因为线程编号可以是2维或3维的，修改代码，重新编译执行，查看结果。

```C
#include <stdio.h>

__global__ void report_in_detail(){
    int ix = blockIdx.x;
    int iy = blockIdx.y;
    int iz = blockIdx.z;

    int jx = threadIdx.x;
    int jy = threadIdx.y;
    int jz = threadIdx.z;

    printf("My group id is (%d,%d,%d), and my thread id is (%d,%d,%d)\n",ix,iy,iz,jx,jy,jz);
}

int main(){
    report_in_detail<<<3,2>>>();
    cudaThreadSynchronize(); // 同步标识。让 CPU 等待 GPU 运行完毕后，再执行后面的语句。试试删除这一行，看看会发生什么
    printf("-----------\n");
    report_in_detail<<<dim3(1,1,3),dim3(1,2,1)>>>();
    return 0;
}
```

为什么 CUDA 中线程需要用到二维或三维分组呢？因为处理图片适合二维编号的线程，渲染真实世界适合三维编号的线程。


### 任务分配

本小节，利用 if 语句，实现简单的任务分配。

```C
#include <stdio.h>

 __global__ void task_allocate(){
    int i = threadIdx.x;
    if(i==0){ // 计算 1+2+3+...+100
        int n = 100;
        int sum = 0;
        while(n>0){
            sum+=n;
            n--;
        }
        printf("1+2+...+100 = %d\n",sum);
    }else if(i==1){ // 计算 10 的阶乘
        int n = 10;
        int factor = 1;
        while(n>0){
            factor*=n;
            n--;
        }
        printf("10! = %d\n",factor);
    }
}

int main(){
    task_allocate<<<1,2>>>();
    return 0;
}
```

运行程序，结果为

<img src="./img/A07TaskAllocate.jpg" width=70%></img>

**试试看** 若将 `<<<1,2>>>` 改为 `<<<1,3>>>` 输出会有变化吗？若改成 `<<<2,2>>>` 呢？

### CPU 和 GPU 数据传输



### 共享内存和线程同步

## CUDA 和科学计算